In [43]:
import numpy as np
import pandas as pd
import random
import torch

In [9]:
from google.cloud import storage
import os

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print(f"Downloaded {source_blob_name} to {destination_file_name}.")

# Example usage
bucket_name = "leo_melon_simple"
blobs_to_download = [
    "exp0901/data/sample100/user_song_dt_000000000000.csv",
    "exp0901/data/sample100/user_song_dt_000000000001.csv",
    "exp0901/data/sample100/user_song_dt_000000000002.csv",
    "exp0901/data/sample100/user_song_dt_000000000003.csv",
    "exp0901/data/sample100/user_song_dt_000000000004.csv",
    "exp0901/data/sample100/user_song_dt_000000000005.csv",
    "exp0901/data/sample100/user_song_dt_000000000006.csv",# Replace with your actual file paths
    "exp0901/data/sample100/song_normalized_data_000000000000.csv"
]
destination_folder = "../data/exp0901/sample100/raw"  # Replace with the local folder path

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for blob_name in blobs_to_download:
    # Create the full local path for each file
    destination_file_path = os.path.join(destination_folder, os.path.basename(blob_name))
    # Download the file from GCS
    download_blob(bucket_name, blob_name, destination_file_path)


Downloaded exp0901/data/sample100/user_song_dt_000000000000.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000000.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000001.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000001.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000002.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000002.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000003.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000003.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000004.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000004.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000005.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000005.csv.
Downloaded exp0901/data/sample100/user_song_dt_000000000006.csv to ../data/exp0901/sample100/raw/user_song_dt_000000000006.csv.
Downloaded exp0901/data/sample100/song_normalized_data_000000000000.csv to ../data/exp0901/sample100/raw

In [1]:
import os
import pandas as pd

fd = "../data/exp0901/sample100/raw"

In [6]:
usersong = pd.concat([pd.read_csv(f"{fd}/user_song_dt_00000000000{i}.csv") for i in range(7)])

In [31]:
snd = pd.read_csv(f"{fd}/song_normalized_data_000000000000.csv")

In [12]:
from torch.utils.data import Dataset, DataLoader

In [41]:
positives = pd.merge(usersong, snd, on=['song_id',"dt"], how="left")

In [22]:
songs_p_user = usersong.groupby(["user_id","dt"])['song_id'].agg(list)

In [82]:
list(snd[['normalized_prev_number_of_plays','transformed_days_from_issue_date']].values)

[array([4.99705797e-04, 9.00601927e-34]),
 array([2.78597094e-04, 8.91640788e-34]),
 array([3.49498266e-04, 8.82768814e-34]),
 array([3.38554063e-05, 8.82768814e-34]),
 array([4.20928568e-05, 8.65288821e-34]),
 array([1.40161382e-04, 9.28029339e-34]),
 array([1.15029349e-04, 9.18795293e-34]),
 array([3.09438413e-03, 1.02563104e-33]),
 array([2.74541146e-03, 1.01542584e-33]),
 array([3.34319327e-03, 1.00532218e-33]),
 array([7.01547614e-06, 1.05686615e-33]),
 array([3.82662335e-06, 1.05686615e-33]),
 array([4.84132310e-04, 1.07821626e-33]),
 array([3.96286081e-04, 1.06748783e-33]),
 array([4.74501295e-04, 1.05686615e-33]),
 array([4.45537146e-04, 1.07821626e-33]),
 array([4.57457758e-04, 1.06748783e-33]),
 array([5.25522940e-04, 1.05686615e-33]),
 array([6.83879207e-05, 8.06789949e-34]),
 array([7.84593248e-05, 7.98762255e-34]),
 array([1.23089718e-04, 7.90814437e-34]),
 array([3.81888983e-04, 1.06748783e-33]),
 array([3.23146033e-04, 1.05686615e-33]),
 array([4.19653027e-04, 1.04635016

In [83]:
pd.Series(data=list(snd[['normalized_prev_number_of_plays','transformed_days_from_issue_date']].values),
          index=snd[['song_id','dt']])

(2241, 2022-01-01)              [0.0004997057965195, 9.00601927212366e-34]
(2241, 2022-01-02)             [0.0002785970940793, 8.916407883107785e-34]
(2241, 2022-01-03)             [0.0003494982659018, 8.827688142310714e-34]
(2307, 2022-01-01)          [3.385540626826076e-05, 8.827688142310714e-34]
(2307, 2022-01-03)          [4.209285684219388e-05, 8.652888205458773e-34]
                                                ...                       
(107659638, 2022-01-03)    [6.377705582150589e-07, 2.6537900154563166e-41]
(107671619, 2022-01-01)       [1.3542162507304304e-06, 0.1422740715865135]
(107676576, 2022-01-02)        [9.308733453726286e-06, 0.1703329888254094]
(107680364, 2022-01-02)        [1.728764784263453e-05, 0.1826835240527346]
(107681124, 2022-01-03)       [1.9133116746451763e-06, 0.1720448638230505]
Length: 338755, dtype: object

In [68]:
snd['dt'].iloc[0]

AttributeError: 'str' object has no attribute 'str'

In [39]:
snd.groupby('dt')['song_id'].agg(list)

dt
2022-01-01    [2241, 2307, 2390, 2397, 2466, 2468, 2498, 252...
2022-01-02    [2241, 2390, 2397, 2466, 2468, 2498, 2528, 253...
2022-01-03    [2241, 2307, 2397, 2459, 2460, 2466, 2468, 249...
Name: song_id, dtype: object

In [77]:
ds.snd.to_dict()

{'normalized_prev_number_of_plays': {(2241, '2022-01-01'): 0.0004997057965195,
  (2241, '2022-01-02'): 0.0002785970940793,
  (2241, '2022-01-03'): 0.0003494982659018,
  (2307, '2022-01-01'): 3.385540626826076e-05,
  (2307, '2022-01-03'): 4.209285684219388e-05,
  (2390, '2022-01-01'): 0.0001401613819505,
  (2390, '2022-01-02'): 0.0001150293491067,
  (2397, '2022-01-01'): 0.003094384132919,
  (2397, '2022-01-02'): 0.0027454114593168,
  (2397, '2022-01-03'): 0.0033431932661633,
  (2459, '2022-01-03'): 7.0154761403656475e-06,
  (2460, '2022-01-03'): 3.8266233492903525e-06,
  (2466, '2022-01-01'): 0.0004841323096361,
  (2466, '2022-01-02'): 0.0003962860813157,
  (2466, '2022-01-03'): 0.000474501295312,
  (2468, '2022-01-01'): 0.0004455371464903,
  (2468, '2022-01-02'): 0.0004574577582974,
  (2468, '2022-01-03'): 0.0005255229399692,
  (2498, '2022-01-01'): 6.838792066188673e-05,
  (2498, '2022-01-02'): 7.845932482426441e-05,
  (2498, '2022-01-03'): 0.0001230897177355,
  (2528, '2022-01-01'):

In [139]:
class myDS(Dataset):
    def __init__(self, usersong, snd):
        # 
        self.positives = pd.merge(usersong, snd, on=['song_id',"dt"], how="left")
        self.songs_p_user = usersong.groupby(["user_id","dt"])['song_id'].agg(list)
        self.snd = pd.Series(data=list(snd[['normalized_prev_number_of_plays','transformed_days_from_issue_date']].values),
          index=snd[['song_id','dt']])
        self.songs_p_dt = snd.groupby('dt')['song_id'].agg(list)
        
    def __len__(self):
        return len(self.positives)
    
    def _get_negative(self, user_id, dt):
        allsongs = self.songs_p_dt[dt]
        listenedsongs = self.songs_p_user[(user_id,dt)]
        notpicked = True
        while notpicked:
            song_id = random.choice(allsongs)
            if song_id not in listenedsongs:
                notpicked = False
        return song_id
    
    def __getitem__(self, index):
        positive = self.positives.iloc[index]
        neg_song_id = self._get_negative(positive.user_id, positive['dt'])
        return {
            'user_id': int(positive.user_id),
            "positive": torch.tensor(positive[['normalized_prev_number_of_plays','transformed_days_from_issue_date']].values),
            "negative": torch.tensor(self.snd[(neg_song_id,positive['dt'])]),
        }

In [140]:
ds = myDS(usersong=usersong,snd=snd)

In [141]:
unique_users = np.unique(usersong.user_id)

In [156]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UserLogisticRegression(nn.Module):
    def __init__(self, unique_users):
        super(UserLogisticRegression, self).__init__()
        # Embedding layer to store coefficients for each user_id
        self.unique_users = {int(u):torch.tensor(i, dtype=torch.long) for i, u in enumerate(unique_users)}
        self.mu = nn.Embedding(len(unique_users), 2)
        self.logvar = nn.Embedding(len(unique_users), 2)
        # Fixed constant
        # self.fixed_constant = 0.

    def forward(self, x, weights):
        logits = torch.sum(weights * x, dim=1)# + self.fixed_constant
        return torch.sigmoid(logits)
    
    def sample_weights(self, user_id):
        mu = self.mu(self.unique_users[user_id])
        logvar = self.logvar(self.unique_users[user_id])
        std = torch.exp(0.5*logvar)
        return mu + std * torch.randn_like(std), mu, logvar
        
    def variational_loss(self, p, y, mu, logvar):
        nll = -torch.distributions.Bernoulli(p).log_prob(y).sum()
        kl = 0.5 * torch.sum(logvar.exp() + mu**2 - 1 - logvar)
        return nll + kl
    
    def train_loss(self, x, y, user_id):
        weights, mu, logvar = self.sample_weights(user_id=user_id)
        p = self.forward(x=x, weights=weights)
        loss = self.variational_loss(p, y, mu, logvar)
        
        

model = UserLogisticRegression(unique_users=unique_users)


In [177]:
torch.randn_like(torch.ones(2))

tensor([ 0.9312, -0.7849])

In [158]:
a['user_id']

58982400

In [159]:
model(a['user_id'], a['positive'].unsqueeze(0))

tensor([0.4999], grad_fn=<SigmoidBackward0>)

In [150]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class UserSongDataset(Dataset):
    def __init__(self, user_song_data, all_songs, num_negatives=1):
        """
        user_song_data: List of tuples (user_id, song_id, x1, x2, y) where y = 1 for positive samples
        all_songs: List or set of all possible song_ids
        num_negatives: Number of negative samples to generate per positive sample
        """
        self.user_song_data = user_song_data
        self.all_songs = list(all_songs)
        self.num_negatives = num_negatives
        
        # Create a dictionary of user_id to set of song_ids for fast lookup
        self.user_song_dict = {}
        for user_id, song_id, _, _, _ in user_song_data:
            if user_id not in self.user_song_dict:
                self.user_song_dict[user_id] = set()
            self.user_song_dict[user_id].add(song_id)
        
        # Precompute the dataset including negative samples
        self.dataset = self._generate_dataset()

    def _generate_dataset(self):
        data = []
        for user_id, song_id, x1, x2, y in self.user_song_data:
            # Add the positive sample
            data.append((user_id, song_id, x1, x2, y))
            
            # Generate negative samples
            for _ in range(self.num_negatives):
                negative_song_id = np.random.choice(self.all_songs)
                while negative_song_id in self.user_song_dict[user_id]:
                    negative_song_id = np.random.choice(self.all_songs)
                data.append((user_id, negative_song_id, x1, x2, 0))  # y=0 for negative sample
        
        return data

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        user_id, song_id, x1, x2, y = self.dataset[idx]
        return torch.tensor(user_id), torch.tensor(song_id), torch.tensor([x1, x2], dtype=torch.float32), torch.tensor(y)

# Example usage
# Suppose we have the following positive data
user_song_data = [
    (0, 101, 0.5, 0.3, 1),
    (0, 102, 0.6, 0.4, 1),
    (1, 101, 0.7, 0.2, 1)
]

# And these are all possible song_ids
all_songs = {101, 102, 103, 104}

# Create the dataset with 2 negative samples per positive sample
dataset = UserSongDataset(user_song_data, all_songs, num_negatives=2)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Iterate over the DataLoader
for batch in dataloader:
    user_ids, song_ids, features, labels = batch
    print("User IDs:", user_ids)
    print("Song IDs:", song_ids)
    print("Features:", features)
    print("Labels:", labels)
    print()


RuntimeError: Could not infer dtype of numpy.int64